In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv(r'E:\Santander Recommendation\train_ver2.csv')
test = pd.read_csv(r'E:\Santander Recommendation\test_ver2.csv')

In [ ]:
train.fecha_dato = pd.to_datetime(train.fecha_dato, format="%Y-%m-%d")
train.fecha_alta = pd.to_datetime(train.fecha_alta, format="%Y-%m-%d")
test.fecha_dato = pd.to_datetime(test.fecha_dato, format="%Y-%m-%d")
test.fecha_alta = pd.to_datetime(test.fecha_alta, format="%Y-%m-%d")

In [ ]:
months = train.fecha_dato.unique()

In [ ]:
train_wnewproducts = pd.DataFrame()

In [ ]:
i = 1
while i < len(months):
    #select sequential months
    new_month = train[train.fecha_dato == months[i]]
    prev_month = train[train.fecha_dato == months[i-1]]
    #select only customers present in both month
    new_month = new_month.loc[new_month['ncodpers'].isin(prev_month.ncodpers)]
    prev_month = prev_month.loc[prev_month['ncodpers'].isin(new_month.ncodpers)]
    #sort by customer id so both months will be ordered the same
    new_month.sort_values(by = 'ncodpers', inplace = True)
    prev_month.sort_values(by = 'ncodpers', inplace = True)
    #pick out product columns
    product_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1',
                'ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1',
                'ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_dela_fin_ult1','ind_deme_fin_ult1',
                'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_nom_pens_ult1',
                'ind_nomina_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
                'ind_recibo_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1']
    #new dataframe with just product
    new_month_products = new_month[product_cols]
    #add in the corresponding customer ids
    new_month_products['ncodpers'] = new_month.ncodpers
    #get rid of index
    new_month_products.reset_index(drop=True,inplace=True)
    #now repeat for the previous month but make ids zero so a subtraction works 
    prev_month_products = prev_month[product_cols]
    prev_month_products['ncodpers'] = prev_month.ncodpers
    prev_month_products.ncodpers = 0
    prev_month_products.reset_index(drop=True,inplace=True)
    #subtract to find the new products
    new_products = new_month_products.subtract(prev_month_products)
    #remove products that the customer dropped fill NA's
    new_products[new_products < 0] = 0
    new_products = new_products.fillna(0)
    #merge the new products features with the month data
    new_products = new_products.merge(new_month.iloc[:,0:24], on='ncodpers')
    #add the interated months into training dataframe
    train_wnewproducts = pd.concat([train_wnewproducts,new_products], axis = 0)
    print("progress = ", float(i/len(months)))
    i += 1

In [ ]:
train_wnewproducts = train_wnewproducts.loc[(train_wnewproducts.iloc[:,0:24] != 0).any(axis=1)]

In [ ]:
final_month = train[train.fecha_dato == '2016-05-28']

In [ ]:
df = pd.concat([train_wnewproducts,test], axis = 0, ignore_index = True)
train = []

In [ ]:
dirtyrows = df[df.ind_empleado.isnull()].index

df = df.drop(dirtyrows, axis = 0)

df.canal_entrada = df.canal_entrada.fillna("Unknown")

df = df.drop("cod_prov", 1)

df.conyuemp = df.conyuemp.fillna("Unknown")

df.loc[df.indrel_1mes == '1', 'indrel_1mes'] = 'Primary'
df.loc[df.indrel_1mes == '1.0', 'indrel_1mes'] = 'Primary'
df.loc[df.indrel_1mes == 1, 'indrel_1mes'] = 'Primary'
df.loc[df.indrel_1mes == 1.0, 'indrel_1mes'] = 'Primary'

df.loc[df.indrel_1mes == '2', 'indrel_1mes'] = 'Co-owner'
df.loc[df.indrel_1mes == '2.0', 'indrel_1mes'] = 'Co-owner'
df.loc[df.indrel_1mes == 2, 'indrel_1mes'] = 'Co-owner'
df.loc[df.indrel_1mes == 2.0, 'indrel_1mes'] = 'Co-owner'

df.loc[df.indrel_1mes == '3', 'indrel_1mes'] = 'Former Primary'
df.loc[df.indrel_1mes == '3.0', 'indrel_1mes'] = 'Former Primary'
df.loc[df.indrel_1mes == 3, 'indrel_1mes'] = 'Former Primary'
df.loc[df.indrel_1mes == 3.0, 'indrel_1mes'] = 'Former Primary'

df.loc[df.indrel_1mes == '4', 'indrel_1mes'] = 'Former Co-owner'
df.loc[df.indrel_1mes == '4.0', 'indrel_1mes'] = 'Former Co-owner'
df.loc[df.indrel_1mes == 4, 'indrel_1mes'] = 'Former Co-owner'
df.loc[df.indrel_1mes == 4.0, 'indrel_1mes'] = 'Former Co-owner'

df.indrel_1mes  = df.indrel_1mes.fillna('Primary')

df.nomprov = df.nomprov.fillna("MADRID")

df.loc[df.renta == '         NA',"renta"] = 0
df.renta = df.renta.astype(float)
df.loc[df.renta == 0, 'renta'] = df[df.renta > 0].groupby('nomprov').renta.transform('median')
df.loc[df.renta.isnull(), "renta"] = df.groupby('nomprov').renta.transform('median')

df.segmento = df[df.renta <= 98000].segmento.fillna("03 - UNIVERSITARIO")
df.segmento = df[df.renta <= 125500].segmento.fillna("02 - PARTICULARES")
df.segmento = df.segmento.fillna("01 - TOP")

df.sexo = df.sexo.fillna("V")

df.tiprel_1mes = df.tiprel_1mes.fillna('I')

df.ind_nomina_ult1 = df.ind_nomina_ult1.fillna(0.0)
df.ind_nom_pens_ult1 = df.ind_nom_pens_ult1.fillna(0.0)

In [ ]:
df.loc[df.antiguedad == -999999, 'antiguedad'] = df[df.antiguedad.astype(float) >= 0].antiguedad.median()

In [ ]:
df['ageGroup'] = (df.age.astype(float) // 10) - 1

df['isSpanish'] = df.pais_residencia.map(lambda x: 1 if x == "ES" else 0)

df['majorCity'] = df.nomprov.map(lambda x: 1 if x == "MADRID" or x == "BARCELONA" else 0)

df['fecha_alta_year'] = pd.DatetimeIndex(df.fecha_alta).year - 1995
df['fecha_dato_year'] = pd.DatetimeIndex(df.fecha_dato).year - 2015
df['fecha_alta_month'] = pd.DatetimeIndex(df.fecha_alta).month - 1
df['fecha_dato_month'] = pd.DatetimeIndex(df.fecha_dato).month - 1

df.antiguedad = df.antiguedad.astype(int)
df['antiguedad_years'] = df.antiguedad // 12

df.loc[df.indrel == 99.0, "indrel"] = 0.0

In [ ]:
df.ult_fec_cli_1t.fillna(0)
df['HAS_ult_fec_cli_1t'] = df.ult_fec_cli_1t.map(lambda x: 1 if type(x)==str else 0)

df = df.drop('ult_fec_cli_1t', 1)

df['rentaGroup'] = df.renta.astype(float) // 50000
df.loc[df.renta >= 1000000, "rentaGroup"] = 20
df.loc[df.renta >= 10000000, "rentaGroup"] = 21

In [ ]:
l = len(train_wnewproducts)- len(dirtyrows)

In [ ]:
train_final, test = df[:l], df[l:]
train_final_training_cols = train_final
train_final_training_cols = train_final_training_cols.drop(product_cols, axis=1)
test = test.drop(product_cols, axis=1)

In [ ]:
print(len(train_final))
print(len(test))

In [ ]:
train_final.ncodpers.isin(test.ncodpers).sum()

In [ ]:
df = pd.concat([train_final_training_cols, test], axis = 0)

In [ ]:
df.pais_residencia = df.pais_residencia.astype('category').cat.codes
df.canal_entrada = df.canal_entrada.astype('category').cat.codes
df.nomprov = df.nomprov.astype('category').cat.codes
final_month.nomprov = final_month.indrel_1mes.astype('category').cat.codes
df = pd.get_dummies(df, columns = ['ind_empleado','sexo','tiprel_1mes','indresi',
                                   'indext','conyuemp','indfall','segmento','indrel_1mes'])

df = df.drop(['fecha_dato', 'fecha_alta'], axis = 1)

In [ ]:
train_final_training_cols, test = df[:l], df[l:]

In [ ]:
labels = train_final[product_cols]
labels['ncodpers'] = train_final.ncodpers
labels = labels.set_index("ncodpers")
stacked_labels = labels.stack()
filtered_labels = stacked_labels.reset_index()
filtered_labels.columns = ["ncodpers", "product", "newly_added"]
filtered_labels = filtered_labels[filtered_labels["newly_added"] == 1]
multiclass_train = filtered_labels.merge(train_final_training_cols, on="ncodpers", how="left")
train_final = multiclass_train.drop_duplicates(multiclass_train, keep='last')
labels_final = train_final['product']
train_final_ncodpers = train_final.ncodpers

train_final = train_final.drop(['ncodpers','newly_added','product'], axis = 1)

In [ ]:
#need to go through and describe each step
dirtyrows = final_month[final_month.ind_empleado.isnull()].index
final_month = final_month.drop(dirtyrows, axis = 0)
final_month.canal_entrada = final_month.canal_entrada.fillna("Unknown")
final_month = final_month.drop("cod_prov", 1)
final_month.conyuemp = final_month.conyuemp.fillna("Unknown")
final_month.loc[final_month.indrel_1mes == '1', 'indrel_1mes'] = 'Primary'
final_month.loc[final_month.indrel_1mes == '1.0', 'indrel_1mes'] = 'Primary'
final_month.loc[final_month.indrel_1mes == 1, 'indrel_1mes'] = 'Primary'
final_month.loc[final_month.indrel_1mes == 1.0, 'indrel_1mes'] = 'Primary'
final_month.loc[final_month.indrel_1mes == '2', 'indrel_1mes'] = 'Co-owner'
final_month.loc[final_month.indrel_1mes == '2.0', 'indrel_1mes'] = 'Co-owner'
final_month.loc[final_month.indrel_1mes == 2, 'indrel_1mes'] = 'Co-owner'
final_month.loc[final_month.indrel_1mes == 2.0, 'indrel_1mes'] = 'Co-owner'
final_month.loc[final_month.indrel_1mes == '3', 'indrel_1mes'] = 'Former Primary'
final_month.loc[final_month.indrel_1mes == '3.0', 'indrel_1mes'] = 'Former Primary'
final_month.loc[final_month.indrel_1mes == 3, 'indrel_1mes'] = 'Former Primary'
final_month.loc[final_month.indrel_1mes == 3.0, 'indrel_1mes'] = 'Former Primary'
final_month.loc[final_month.indrel_1mes == '4', 'indrel_1mes'] = 'Former Co-owner'
final_month.loc[final_month.indrel_1mes == '4.0', 'indrel_1mes'] = 'Former Co-owner'
final_month.loc[final_month.indrel_1mes == 4, 'indrel_1mes'] = 'Former Co-owner'
final_month.loc[final_month.indrel_1mes == 4.0, 'indrel_1mes'] = 'Former Co-owner'
final_month.indrel_1mes = final_month.indrel_1mes.fillna('Primary')
final_month.nomprov = final_month.nomprov.fillna("MADRID")
final_month.renta = final_month.renta.astype(float)
final_month.loc[final_month.renta.isnull(), "renta"] = final_month.groupby('nomprov').renta.transform('median')
final_month.segmento = final_month[final_month.renta <= 98000].segmento.fillna("03 - UNIVERSITARIO")
final_month.segmento = final_month[final_month.renta <= 125500].segmento.fillna("02 - PARTICULARES")
final_month.segmento = final_month.segmento.fillna("01 - TOP")
final_month.sexo = final_month.sexo.fillna("V")
final_month.tiprel_1mes = final_month.tiprel_1mes.fillna('I')
final_month.ind_nomina_ult1 = final_month.ind_nomina_ult1.fillna(0.0)
final_month.ind_nom_pens_ult1 = final_month.ind_nom_pens_ult1.fillna(0.0)
final_month.loc[final_month.antiguedad == -999999, 'antiguedad'] = final_month[final_month.antiguedad >= 0].antiguedad.median()
final_month.age = final_month.age.astype(int)
final_month.loc[final_month.age < 20,"age"] = 19
final_month.loc[final_month.age > 90,"age"] = 91
final_month['ageGroup'] = (final_month.age // 10) - 1
final_month['isSpanish'] = final_month.pais_residencia.map(lambda x: 1 if x == "ES" else 0)
final_month['majorCity'] = final_month.nomprov.map(lambda x: 1 if x == "MADRID" or x == "BARCELONA" else 0)
final_month['fecha_alta_year'] = pd.DatetimeIndex(final_month.fecha_alta).year - 1995
final_month['fecha_dato_year'] = pd.DatetimeIndex(final_month.fecha_dato).year - 2015
final_month['fecha_alta_month'] = pd.DatetimeIndex(final_month.fecha_alta).month - 1
final_month['fecha_dato_month'] = pd.DatetimeIndex(final_month.fecha_dato).month - 1
final_month.antiguedad = final_month.antiguedad.astype(int)
final_month['antiguedad_years'] = final_month.antiguedad // 12
final_month.loc[final_month.indrel == 99.0, "indrel"] = 0.0
final_month.ult_fec_cli_1t.fillna(0)
final_month['HAS_ult_fec_cli_1t'] = final_month.ult_fec_cli_1t.map(lambda x: 1 if type(x) == str else 0)
final_month = final_month.drop('ult_fec_cli_1t', 1)
final_month['rentaGroup'] = final_month.renta.astype(float) // 50000
final_month.loc[final_month.renta >= 1000000, "rentaGroup"] = 20
final_month.loc[final_month.renta >= 10000000, "rentaGroup"] = 21
final_month_training_cols = final_month
final_month_training_cols = final_month_training_cols.drop(product_cols, axis=1)
final_month.pais_residencia = final_month.pais_residencia.astype('category').cat.codes
final_month.canal_entrada = final_month.canal_entrada.astype('category').cat.codes
final_month.nomprov = final_month.nomprov.astype('category').cat.codes
final_month.nomprov = final_month.indrel_1mes.astype('category').cat.codes
final_month = pd.get_dummies(final_month, columns = ['ind_empleado','sexo','tiprel_1mes','indresi',
                                   'indext','conyuemp','indfall','segmento','indrel_1mes'])
final_month = final_month.drop(['fecha_dato', 'fecha_alta'], axis = 1)

In [ ]:
labels_final_month = final_month[product_cols]
labels_final_month['ncodpers'] = final_month.ncodpers
labels_final_month = labels_final_month.set_index("ncodpers")
stacked_labels_final_month = labels_final_month.stack()
filtered_labels_final_month = stacked_labels_final_month.reset_index()
filtered_labels_final_month.columns = ["ncodpers", "product", "newly_added"]
filtered_labels_final_month = filtered_labels_final_month[filtered_labels_final_month["newly_added"] == 1]
multiclass_final_month = filtered_labels_final_month.merge(final_month_training_cols, on="ncodpers", how="left")
final_month = multiclass_final_month.drop_duplicates(multiclass_final_month, keep='last')
labels_final_month_final = final_month['product']
final_month_ncodpers = final_month.ncodpers
final_month = final_month.drop(['ncodpers','newly_added','product'], axis = 1)

In [ ]:
print(len(train_final))
print(len(labels_final))
print(len(final_month))
print(len(labels_final_month_final))

In [ ]:
labels_final.value_counts()

In [ ]:
labels_final = labels_final.astype('category').cat.codes

In [ ]:
test = test.drop('ncodpers', axis = 1)

In [ ]:
from CFModel import CFModel

In [ ]:
from surprise import Reader, Dataset

In [ ]:
train = pd.read_csv(r'E:\Santander Recommendation\train_ver2.csv')
lastmonth = train.loc[train['fecha_dato']=='2016-05-28']
train=[]
lastmonthlabels = lastmonth[product_cols]
lastmonthlabels['ncodpers'] = lastmonth.ncodpers
lastmonthlabels = lastmonthlabels.set_index('ncodpers')
stacked_lastmonthlabels = lastmonthlabels.stack()
filtered_lastmonthlabels = stacked_lastmonthlabels.reset_index()
filtered_lastmonthlabels.columns = ["ncodpers", "product", "is_owned"]

In [ ]:
reader = Reader(line_format='user item rating', rating_scale=(0,1))
data = Dataset.load_from_df(filtered_lastmonthlabels, reader=reader)


In [ ]:
collabfilter = CFModel(n_users, m_items, kfactors)
collabfilter.compile(loss='mse', optimizer='adamax')

In [ ]:
import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

In [ ]:
y = to_categorical(labels_final)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_final, y, test_size=0.2, random_state=21)

In [ ]:
X_train = normalize(X_train)
X_test = normalize(X_test)

In [ ]:
X_train.shape

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(512,activation='relu',input_shape=(45,)))

In [ ]:
model.add(Dropout(0.2))

In [ ]:
model.add(Dense(512,activation='relu'))

In [ ]:
model.add(Dropout(0.2))

In [ ]:
model.add(Dense(24,activation='sigmoid'))

In [ ]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
early_stopping_monitor = EarlyStopping(patience=1)

In [ ]:
model_1_training = model.fit(X_train,y_train,validation_split=0.3,epochs=3, batch_size = 32,verbose =1,
                              validation_data = (X_test, y_test), callbacks=[early_stopping_monitor])

In [ ]:
ynew = model.predict(X_test)
ynew[0]

In [ ]:
score = model.evaluate(X_test, y_test,batch_size=32)
score

In [ ]:
labels_final_month_final_cat = labels_final_month_final.astype('category').cat.codes

In [ ]:
used_products = pd.DataFrame()
used_products['product'] = labels_final_month_final_cat
used_products['ncodpers'] = final_month_ncodpers
used_products = used_products.drop_duplicates(keep = 'last')

In [ ]:
#create a dictionary to store each product a customer already has
used_recommendation_products = {}
product_cols = np.array(product_cols)
#iterate through used_products and add each one to used_recommendation_products
for idx,row_val in used_products.iterrows():
    used_recommendation_products.setdefault(row_val['ncodpers'],[]).append(product_cols[row_val['product']])

In [ ]:
used_recommendation_products[15889]

In [ ]:
modeltest = normalize(test)

In [ ]:
modelpreds = model.predict(modeltest)

In [ ]:
modelpreds

In [ ]:
pred = np.argsort(modelpreds, axis=1)
pred = np.fliplr(pred)

In [ ]:
pred[0]

In [ ]:
test_ids = np.array(pd.read_csv(r'E:\Santander Recommendation\test_ver2.csv',usecols=['ncodpers'])['ncodpers'])
product_cols = np.array(product_cols)
final_preds = []
#iteratively create the final rank ordered recommendations for each user, exluding ones the customer already has
for idx,predicted in enumerate(pred):
    ids = test_ids[idx]
    top_product = product_cols[predicted]
    used_products = used_recommendation_products.get(ids,[])
    new_top_product = []
    for product in top_product:
        if product not in used_products:
            new_top_product.append(product)
        if len(new_top_product) == 7:
            break
    final_preds.append(' '.join(new_top_product))

In [ ]:
submission = pd.DataFrame({'ncodpers':test_ids,'added_products':final_preds})
submission.to_csv('submission.csv',index=False)